In [ ]:
import os
import cv2
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import skimage as sk
import skimage.io as skio
import json

In [ ]:
# Import images
theo = sk.img_as_float(np.array(skio.imread("theo_nobg.png")))[:,:,:-1]
magnus = sk.img_as_float(np.array(skio.imread("magnus_nobg.png")))[:,:,:-1]
figure,axes = plt.subplots(2)
axes[0].imshow(theo)
axes[1].imshow(magnus)
plt.show()

# Defining Correspondences

In [ ]:
f = open("theo_magnus.json")
points = json.load(f)
theo_points = np.array(points['im1Points'])
theo_points[:,[0,1]] = theo_points[:,[1,0]]
magnus_points = np.array(points['im2Points'])
magnus_points[:,[0,1]] = magnus_points[:,[1,0]]

In [ ]:
avg_points = 0.5*theo_points + 0.5*magnus_points  
triangulation = sp.spatial.Delaunay(avg_points)

In [ ]:
#plt.figure(figsize = (1,1))
plt.imshow(theo)
plt.triplot(theo_points[:,1],theo_points[:,0],triangulation.simplices)
plt.plot(theo_points[:,1],theo_points[:,0],'o')
plt.show()

In [ ]:
plt.imshow(theo)
plt.triplot(avg_points[:,1],avg_points[:,0],triangulation.simplices)
plt.plot(avg_points[:,1],avg_points[:,0],'o')
plt.show()

In [ ]:
plt.imshow(magnus)
plt.triplot(magnus_points[:,1],magnus_points[:,0],triangulation.simplices)
plt.plot(magnus_points[:,1],magnus_points[:,0],'o')
plt.show()

In [ ]:
plt.imshow(magnus)
plt.triplot(avg_points[:,1],avg_points[:,0],triangulation.simplices)
plt.plot(avg_points[:,1],avg_points[:,0],'o')
plt.show()

# Computing the Mid-way Face

In [ ]:
def computeInverseAffine(tri1,tri2):
    mat1 = np.hstack((tri1, [[1],[1],[1]]))
    mat2 = np.hstack((tri2, [[1],[1],[1]]))
    return np.linalg.solve(mat2,mat1)

In [ ]:
def warp(orig_img, orig_triangles, target_triangles):
    warp_img = np.ones(orig_img.shape)
    img_points = np.array(list(np.ndindex(orig_img[:,:,0].shape)))
    img_vals = np.array(list(zip(orig_img[:,:,0].flatten(),orig_img[:,:,1].flatten(),orig_img[:,:,2].flatten())))
    
    for orig_tri, target_tri in zip(orig_triangles, target_triangles):
        transform_matrix = computeInverseAffine(orig_tri, target_tri)
        target_points = np.vstack(sk.draw.polygon(target_tri[:,0], target_tri[:,1])).T
        target_transform = (np.hstack((target_points,np.ones((target_points.shape[0],1)))) @ transform_matrix)[:,:2]
        data = sp.interpolate.griddata(img_points, img_vals, target_transform, method="nearest")
        #print(target_points)
        for j in range(target_points.shape[0]):
            x,y = target_points[j]
            warp_img[x,y] = data[j]
    return warp_img                
                

In [ ]:
theo_triangles = theo_points[triangulation.simplices]
magnus_triangles = magnus_points[triangulation.simplices]
avg_triangles = avg_points[triangulation.simplices]
theo_to_avg_warp = warp(theo, theo_triangles, avg_triangles)
magnus_to_avg_warp = warp(magnus, magnus_triangles, avg_triangles)

In [ ]:
plt.imshow(theo_to_avg_warp)
skio.imsave("theo_warped_midway.jpg", sk.img_as_ubyte(theo_to_avg_warp))

In [ ]:
mid = 0.5*theo_to_avg_warp + 0.5*magnus_to_avg_warp
plt.imshow(mid)
skio.imsave("theo_magnus_midway.jpg", sk.img_as_ubyte(mid))

In [ ]:
plt.imshow(magnus_to_avg_warp)
skio.imsave("magnus_warped_midway.jpg", sk.img_as_ubyte(magnus_to_avg_warp))

# Morphing Sequence

In [ ]:
def morph(im1,im2,im1_pts,im2_pts,tri,warp_frac,dissolve_frac):
    warped_points = (1-warp_frac)*im1_pts + warp_frac*im2_pts
    im1_warp = warp(im1,im1_pts[tri],warped_points[tri])
    im2_warp = warp(im2,im2_pts[tri],warped_points[tri])
    dissolved = (1-dissolve_frac)*im1_warp + dissolve_frac*im2_warp
    return dissolved

In [ ]:
def morph_sequence(im1,im2,im1_pts,im2_pts, filename):
    avg_pts = 0.5*im1_pts + 0.5*im2_pts
    tri = sp.spatial.Delaunay(avg_pts).simplices
    
    num_frames = 50
    parameters = np.linspace(0,1,num = num_frames)
    total = 0
    for p in parameters:
        im_morphed = morph(im1,im2,im1_pts,im2_pts,tri,p,p)
        fname = "{}_{}.jpg".format(filename, str(total))
        print("Saving " + fname)
        skio.imsave(fname, sk.img_as_ubyte(im_morphed))
        total += 1

In [ ]:
morph_sequence(theo,magnus,theo_points,magnus_points, "theo_magnus")

In [ ]:
def make_video(name, images):
    video_writer = cv2.VideoWriter(name, cv2.VideoWriter_fourcc(*'mp4v'),15,(images[0].shape[1], images[0].shape[0]))
    for im in images:
        video_writer.write(im)
    video_writer.release()    

In [ ]:
images = []
for i in range(50):
    img = cv2.imread("./theo_to_magnus_nobg/theo_magnus_" + str(i) + ".jpg")
    images.append(img)
make_video("theo_to_magnus.mp4", images + list(reversed(images)))    

# Mean Face of Population

In [ ]:
img = sk.img_as_float(skio.imread("./dane_faces/33-1m.jpg"))
width = img.shape[1]
height = img.shape[0]
# f = open("./dane_faces/33-1m.asf",'r')
# txt = f.read()
# data = np.array([s.split('\t') for s in txt.split('\n')[16:-6]])[:,2:4].astype(float)
# data[:,0], data[:,1] = data[:,0]*width, data[:,1]*height
# data = np.round(data).astype(int)
# data = np.vstack([data, [0,0], [0,height-1],[width-1,0], [width-1,height-1]])

In [ ]:
# Collect correspondence points for each image
corr_pts = {}
for filename in os.listdir("./dane_faces"):
    if filename.endswith("1m.asf") or filename.endswith("1f.asf"):
        f = open("./dane_faces/" + filename,'r')
        txt = f.read()
        data = np.array([s.split('\t') for s in txt.split('\n')[16:-6]])[:,2:4].astype(float)
        data[:,0], data[:,1] = data[:,0]*width, data[:,1]*height
        data = np.round(data).astype(int)
        data = np.vstack([data, [0,0], [0,height-1],[width-1,0], [width-1,height-1]])
        # Switch columns
        data[:,[0,1]] = data[:,[1,0]]
        corr_pts[filename[:-4]] = data

# Find average shape
danes_avg_shape = np.mean([corr_pts[filename] for filename in corr_pts.keys()],0)
danes_avg_shape = np.round(danes_avg_shape).astype(int)

In [ ]:
danes_avg_tri = sp.spatial.Delaunay(danes_avg_shape)

In [ ]:
plt.imshow(img)
plt.triplot(danes_avg_shape[:,1],danes_avg_shape[:,0],danes_avg_tri.simplices)
plt.plot(danes_avg_shape[:,1],danes_avg_shape[:,0],'o')
plt.show()

In [ ]:
triangles = corr_pts["33-1m"][danes_avg_tri.simplices]
im_warp = warp(img, triangles, danes_avg_shape[danes_avg_tri.simplices])
plt.imshow(im_warp)

## Morph each face into the average face

In [ ]:
if not os.path.isdir("warped_danes"):
    os.mkdir("warped_danes")

In [ ]:
for filename in os.listdir("./dane_faces"):
    if filename.endswith("1m.jpg") or filename.endswith("1f.jpg"):
        dane_img = sk.img_as_float(skio.imread("./dane_faces/" + filename))
        dane_tris = corr_pts[filename[:-4]][danes_avg_tri.simplices]
        dane_warp = warp(dane_img, dane_tris, danes_avg_shape[danes_avg_tri.simplices])
        print("Saving Warped " + filename)
        fname = "warped_danes/" + filename[:-4] + "_warped.jpg"
        plt.imsave(fname, sk.img_as_ubyte(dane_warp))

In [ ]:
# Compute average face
warped_faces = []
for filename in os.listdir("./warped_danes"):
    if filename.endswith(".jpg"):
        img = sk.img_as_float(skio.imread("./warped_danes/" + filename))
        warped_faces.append(img)
average_dane_face = np.mean(warped_faces,0)

In [ ]:
fig, axes = plt.subplots(5,8, figsize = (25,13), constrained_layout=True)
count = 0
for i in range(5):
    for j in range(8):
        #axes[i][j] = fig.add_subplot(gs[r,c[)
        axes[i][j].imshow(warped_faces[count])
        axes[i][j].axis('off')
        axes[i][j].set_aspect('auto')
        count += 1
plt.show()        

In [ ]:
dane_faces = []
for filename in os.listdir("./dane_faces"):
    if filename.endswith("1m.jpg") or filename.endswith("1f.jpg"):
        dane_img = sk.img_as_float(skio.imread("./dane_faces/" + filename))
        dane_faces.append(dane_img)

In [ ]:

fig, axes = plt.subplots(5,8, figsize = (25,13), constrained_layout=True)
count = 0
for i in range(5):
    for j in range(8):
        axes[i][j].imshow(dane_faces[count])
        axes[i][j].axis('off')
        axes[i][j].set_aspect('auto')
        count += 1
plt.show()  

In [ ]:
skio.imsave("dane1.jpg", sk.img_as_ubyte(dane_faces[0]))
skio.imsave("dane1_warped.jpg", sk.img_as_ubyte(warped_faces[0]))

In [ ]:
skio.imsave("dane2.jpg", sk.img_as_ubyte(dane_faces[29]))
skio.imsave("dane2_warped.jpg", sk.img_as_ubyte(warped_faces[29]))

In [ ]:
plt.imshow(average_dane_face)
skio.imsave("average_dane_face.jpg", sk.img_as_ubyte(average_dane_face))

In [ ]:
plt.imshow(average_dane_face)
plt.triplot(danes_avg_shape[:,1],danes_avg_shape[:,0],danes_avg_tri.simplices)
plt.plot(danes_avg_shape[:,1],danes_avg_shape[:,0],'o')
# for i,point in enumerate(danes_avg_shape):
#     plt.annotate(str(i+1),(point[1],point[0]), color='white', fontsize=5,
#                  ha='center', va='center', weight='bold')
plt.show()

## Warp my face into average shape and vice versa

In [ ]:
theop = sk.img_as_float(skio.imread("theop.jpg"))
theop_pts = np.array(json.load(open("dane_theop.json"))['im2Points'])
theop_pts[:,[0,1]] = theop_pts[:,[1,0]]
theop_tri = sp.spatial.Delaunay(theop_pts)
plt.imshow(theop)

In [ ]:
plt.imshow(theop)
plt.triplot(danes_avg_shape[:,1],danes_avg_shape[:,0],danes_avg_tri.simplices)
plt.plot(danes_avg_shape[:,1],danes_avg_shape[:,0],'o')
for i,point in enumerate(danes_avg_shape):
    plt.annotate(str(i+1),(point[1],point[0]), color='white', fontsize=5,
                 ha='center', va='center', weight='bold')
plt.show()

In [ ]:
theop_pts.shape

In [ ]:
theop_to_avg = warp(theop, theop_pts[danes_avg_tri.simplices], danes_avg_shape[danes_avg_tri.simplices])

In [ ]:
plt.imshow(theop_to_avg)
skio.imsave("theop_to-avg.jpg",sk.img_as_ubyte(theop_to_avg))

In [ ]:
# Avg Face to my geometry
avg_to_theop = warp(average_dane_face, danes_avg_shape[theop_tri.simplices], theop_pts[theop_tri.simplices])

In [ ]:
plt.imshow(avg_to_theop)
skio.imsave("avg_to_theop.jpg", sk.img_as_ubyte(avg_to_theop))

# Caricature (Extrapolate from population mean)

In [ ]:
diff = theop_pts - danes_avg_shape
newShape = theop_pts + 1*diff
newShape_tri = sp.spatial.Delaunay(newShape)
caricature = warp(theop, theop_pts[newShape_tri.simplices], newShape[newShape_tri.simplices])

In [ ]:
plt.imshow(caricature)
skio.imsave("theop_caricature.jpg",sk.img_as_ubyte(caricature))

# Bells and Whistles

In [ ]:
# Import images
theo = sk.img_as_float(np.array(skio.imread("theo_crop.jpg")))
woman = sk.img_as_float(np.array(skio.imread("woman.jpg")))
figure,axes = plt.subplots(2)
axes[0].imshow(theo)
axes[1].imshow(woman)
plt.show()

In [ ]:
f = open("theo_crop_woman.json")
points = json.load(f)
theo_points = np.array(points['im1Points'])
theo_points[:,[0,1]] = theo_points[:,[1,0]]
woman_points = np.array(points['im2Points'])
woman_points[:,[0,1]] = woman_points[:,[1,0]]

In [ ]:
theo_tri = sp.spatial.Delaunay(woman_points)
woman_tri = sp.spatial.Delaunay(theo_points)

In [ ]:
plt.imshow(theo)
plt.triplot(theo_points[:,1],theo_points[:,0],theo_tri.simplices)
plt.plot(theo_points[:,1],theo_points[:,0],'o')
plt.show()

In [ ]:
plt.imshow(woman)
plt.triplot(woman_points[:,1],woman_points[:,0],woman_tri.simplices)
plt.plot(woman_points[:,1],woman_points[:,0],'o')
plt.show()

In [ ]:
# Change shape
theo_to_woman_shape = warp(theo, theo_points[woman_tri.simplices], woman_points[woman_tri.simplices])
plt.imshow(theo_to_woman_shape)

In [ ]:
skio.imsave("theo_woman_shape.jpg", sk.img_as_ubyte(theo_to_woman_shape))

In [ ]:
# Change appearance
#woman_to_theo = warp(woman, woman_points[theo_tri.simplices], theo_points[theo_tri.simplices])
theo_to_woman_appearance = 0.3*theo + 0.7*woman_to_theo
plt.imshow(theo_to_woman_appearance)

In [ ]:
skio.imsave("theo_to_woman_appearance.jpg", sk.img_as_ubyte(theo_to_woman_appearance))

In [ ]:
# Change both
theo_to_woman = morph(theo,woman,theo_points, woman_points,theo_tri.simplices,0.75,0.5)
plt.imshow(theo_to_woman)
skio.imsave("theo_to_woman.jpg", sk.img_as_ubyte(theo_to_woman))